## Polytech Paris Saclay | 3ème année


## TP2 Calcul scientifique: la factorisation LU

- rappel

La factorisation LU consiste à factoriser une matrice A en un produit de deux matrices: une matrice triangulaire inférieure L et une matrice triangulaire supérieure U telles que A = L\*U. Le système Ax=b devient alors LUx=b. Une telle factorisation permet de résoudre plus facilement le nouveau système: deux résolutions successives: une par descente et une par remontée.

       Algorithm 1: In place LU factorisation without pivoting
       Require: A is a n*n matrix
        1: for k <- 1 to n-1 do
        2:    for i <- k+1 to n do
        3:       A(i,j) <- A(i,j)/A(k,k)
        4:    end for
        5:    for i <- k+1 to n do
        6:       for j <- k+1 to n do
        7:          A(i,j) <- A(i,j) - A(i,k)*A(k,j)
        8:       end for
        9:    end for
        10: end for
   

- écrire une fonction *monLU(A::Matrix{Float64})* qui met en oeuvre la factorisation LU sans pivotage.

In [5]:
using LinearAlgebra

function monLU(A)
   
   #initiliasation 
    AA = copy(A)
    n,m = size(AA)
    
    #début de l'agorithme
    for k =1:n-1
        for i = k+1:n
            AA[i,k] = AA[i,k]/AA[k,k]
        end
        for i = k+1:n
            for j = k+1:n
                AA[i,j] = AA[i,j] - AA[i,k]*AA[k,j]  
            end
        end
    end
    
    #détermination de la matrice L et U 
    Identite = Matrix{Float64}(I, n,n)
    L = tril(AA,-1)+Identite
    U = triu(AA)
    return L,U
end         

monLU (generic function with 1 method)

In [28]:
#création des tests
A = [1 1 2; 
    -1 -2 3;
    3 -7 4.]

b = [8,1,10.]

L,U = monLU(A)
L

3×3 Array{Float64,2}:
  1.0   0.0  0.0
 -1.0   1.0  0.0
  3.0  10.0  1.0

In [29]:
U

3×3 Array{Float64,2}:
 1.0   1.0    2.0
 0.0  -1.0    5.0
 0.0   0.0  -52.0

In [30]:
L*U

3×3 Array{Float64,2}:
  1.0   1.0  2.0
 -1.0  -2.0  3.0
  3.0  -7.0  4.0

- écrire une fonction *descente(L::Matrix{Float64}, b::Vector{Float64})* qui retourne *z* en mettant en oeuvre la résolution par decente d'un système triangulaire inférieur.

In [31]:
function descente(L,b)

    n,m = size(L)
    x = zeros(Int, n)
    t = 0
    
    for i = 1:n
        for j = 1:i-1
            t = t+L[i,j]*x[j]
        end
        x[i] = b[i]-t
        t = 0
    end
    return x
end


descente (generic function with 1 method)

In [32]:
y = descente(L,b)

3-element Array{Int64,1}:
    8
    9
 -104

In [33]:
L*y

3-element Array{Float64,1}:
  8.0
  1.0
 10.0

- écrire une fonction *remontee(U::Matrix{Float64}, z::Vector{Float64})* qui retourne *x* en mettant en oeuvre la résolution par remontée d'un système triangulaire supérieur.

In [34]:
function remonte(U, z)
    x = zeros(1, size(b,1))

    for i in 1:size(z,1)
        sum = 0
        for j in i+1:size(z,1)
            sum += x[j] * U[i,j]
        end
        x[i] = (z[i] - sum) / U[i,i]
    end
    return x
end


remonte (generic function with 1 method)

In [35]:
remonte(U,y)

1×3 Array{Float64,2}:
 8.0  -9.0  2.0

- écrire une fonction *monPivotLu(A::Matrix{Float64})* qui met en oeuvre la factorisation LU avec pivotage partiel.

In [40]:
function monPivotLu(A::Matrix{Float64}) # retourne perm, UP et LP 
   #initiliasation 
    AA = copy(A)
    n = size(A,1) #pour récupérer les dimensions de la matrice 
    Identite = Matrix{Float64}(I, n,n)
    Q = Matrix{Float64}(I, n,n)

    for k = 1:(n-1)
        imax=1 #ligne du pivot max
        pivot=abs(A[k,k])

        for i = k+1:n
            if abs(AA[i,k]) > pivot
                pivot = abs(AA[i,k])
                imax = i
            end
        end
        AA[k,:],AA[imax,:] = AA[imax,:],AA[k,:] #on interverti deux lignes entre elles
        Q[k,:],Q[imax,:] = Q[imax,:],Q[k,:]

        for i = k+1:n 
            AA[i,k] = AA[i,k]/AA[k,k]
        end
        for i = k+1:n
            for j = k+1:n
                AA[i,j] = AA[i,j] - AA[i,k]*AA[k,j]  
            end
        end
    end
    
    #détermination de la matrice L et U 
    
    LP = tril(AA,-1)+Identite
    UP = triu(AA)
    return L,U,Q
end

monPivotLu (generic function with 2 methods)

In [41]:
L,U,Q = monPivotLu(A)

([1.0 0.0 0.0; -1.0 1.0 0.0; 3.0 10.0 1.0], [1.0 1.0 2.0; 0.0 -1.0 5.0; 0.0 0.0 -52.0], [0.0 0.0 1.0; 1.0 0.0 0.0; 0.0 1.0 0.0])

- tester vos fonctions en résolvant les systèmes d'équations suivants:
  1. premier système à résoudre:
$$\begin{align*} 
x_1 + x_2 + 2x_3 &=  8 \\ 
-x_1 - 2x_2 + 3x_3 &=  1 \\
3x_1 - 7x_2 + 4x_3 &= 10
\end{align*}$$
  1. deuxième système à résoudre:
$$\begin{align*} 
x_1 - x_2 + 2x_3 - x_4 &= -1 \\ 
2x_1 + x_2 - 2x_3 - 2x_4 &= -2 \\
-x_1 + 2x_2 - 4x_3 + x_4 &=  1 \\
3x_1 - 3x_4 &= -3
\end{align*}$$

In [42]:
  #2
  A2 = [1. -1 2 -1;
        2 1 -2 -2;
        -1 2 -4 1;
        3 0 0 -3]
  b2 = [-1.;2;1;-3]

println(A2)

L2,U2,Q2 = monPivotLu(A2)
y2 = descente(L2,b2)
x2 = remonte(U2,y2)
print(" les solutions du systèmes sont : ")
println(x2)
 

[1.0 -1.0 2.0 -1.0; 2.0 1.0 -2.0 -2.0; -1.0 2.0 -4.0 1.0; 3.0 0.0 0.0 -3.0]
 les solutions du systèmes sont : [-1.0 -1.0 0.11538461538461539]


## fin du TP2

Soit **f** la fonction de la variable réelle **x** telle que $$f(x)=x^3-4x+1.$$ Calculer la fonction dérivée **df** de **f** (à la main).